## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-14-10-49-43 +0000,bbc,Netanyahu is only obstacle to bringing hostage...,https://www.bbc.com/news/articles/c1dqrg32zd9o...
1,2025-09-14-10-15-42 +0000,bbc,Pot breaks as Nigerian chef attempts largest p...,https://www.bbc.com/news/articles/c306m1j542po...
2,2025-09-14-10-00-18 +0000,wapo,Political turbulence engulfs the Fed as Trump ...,https://www.washingtonpost.com/business/2025/0...
3,2025-09-14-10-00-18 +0000,nypost,NY’s parole-reform law — backed by Mamdani — a...,https://nypost.com/2025/09/14/us-news/nys-paro...
4,2025-09-14-10-00-14 +0000,wapo,‘Can we talk about RFK Jr.?’ Inside the chill ...,https://www.washingtonpost.com/health/2025/09/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2333/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
154,kirk,27
153,charlie,27
18,trump,22
32,new,13
91,california,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
105,2025-09-13-20-58-19 +0000,nypost,"Jasmine Crockett denies ‘wannabe Hitler,’ left...",https://nypost.com/2025/09/13/us-news/jasmine-...,91
117,2025-09-13-19-19-12 +0000,nypost,Nearly $3M raised for Charlie Kirk’s family fo...,https://nypost.com/2025/09/13/business/new-fun...,89
135,2025-09-13-17-24-02 +0000,nyt,Trump Escalates Attacks on Political Opponents...,https://www.nytimes.com/2025/09/13/us/politics...,87
91,2025-09-13-22-53-00 +0000,nypost,NY and NJ skipped flag order for Charlie Kirk ...,https://nypost.com/2025/09/13/us-news/ny-and-n...,84
97,2025-09-13-22-23-48 +0000,nypost,Charlie Kirk’s accused assassin Tyler Robinson...,https://nypost.com/2025/09/13/us-news/accused-...,83


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
105,91,2025-09-13-20-58-19 +0000,nypost,"Jasmine Crockett denies ‘wannabe Hitler,’ left...",https://nypost.com/2025/09/13/us-news/jasmine-...
59,45,2025-09-14-03-00-00 +0000,wsj,Israeli Prime Minister Benjamin Netanyahu and ...,https://www.wsj.com/politics/national-security...
16,37,2025-09-14-09-30-00 +0000,wsj,In the weeks since President Trump’s 50% tarif...,https://www.wsj.com/economy/trade/global-trade...
101,34,2025-09-13-21-32-26 +0000,nypost,Israel bombs Hamas operatives in another Gaza ...,https://nypost.com/2025/09/13/world-news/israe...
62,31,2025-09-14-02-56-48 +0000,nypost,Warner Bros. Discovery shares spike as CEO Dav...,https://nypost.com/2025/09/13/business/warner-...
97,29,2025-09-13-22-23-48 +0000,nypost,Charlie Kirk’s accused assassin Tyler Robinson...,https://nypost.com/2025/09/13/us-news/accused-...
126,26,2025-09-13-18-45-30 +0000,nypost,"Russia kills 4 in heavy blitz on Donetsk, Ukra...",https://nypost.com/2025/09/13/world-news/russi...
70,24,2025-09-14-01-00-00 +0000,wsj,The unemployment rate is low. But a measure th...,https://www.wsj.com/economy/jobs/underemployed...
129,24,2025-09-13-18-27-38 +0000,latimes,"Lawyers fear 1,000 children from Central Ameri...",https://www.latimes.com/california/story/2025-...
147,24,2025-09-13-15-08-00 +0000,wsj,Fed Reserve governor Lisa Cook described one o...,https://www.wsj.com/economy/central-banking/do...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
